Questions Formule Un 

In [51]:
# Librairies
import pandas as pd
import os
import csv
from datetime import datetime

In [66]:
# Chargement des tables utiles
laps = pd.read_csv(os.path.join("donnees_formule_un", "lap_times.csv"))
laps.columns = laps.columns.str.strip()

races = pd.read_csv(os.path.join("donnees_formule_un", "races.csv"))
races.columns = races.columns.str.strip()

drivers = pd.read_csv(os.path.join("donnees_formule_un", "drivers.csv")\
    , on_bad_lines='skip')
drivers.columns = drivers.columns.str.strip()
drivers["forename"] = drivers["forename"].str.replace('"', '').str.strip()
drivers["surname"] = drivers["surname"].str.replace('"', '').str.strip()
drivers["nom_complet"] = drivers["forename"] + " " + drivers["surname"]

results = pd.read_csv(os.path.join("donnees_formule_un", "results.csv"))
results.columns = results.columns.str.strip()

pit = pd.read_csv(os.path.join("donnees_formule_un", "pit_stops.csv"))
pit.columns = pit.columns.str.strip()

circuits = pd.read_csv(os.path.join("donnees_formule_un", "races.csv"))
circuits.columns = circuits.columns.str.strip()

Question 1 : Quels sont les pilotes qui ont gagné au moins 30 courses ? 

Question 2 : Quel est le classement des pilotes à l'issue de la saison 2023 ? 

Question 3 : 

Question 4 :  Quels sont les meilleurs temps d'un tour de circuit (best lap time) pour chaque circuit, en 2023 ?

In [ ]:
# Fusion des tables
fusion = pd.merge(laps, races, on='raceId')
fusion.columns = fusion.columns.str.strip()
# Extraire les années des courses
annees_courses = fusion[['year', 'raceId', 'name']].drop_duplicates()
# Extraire le temps le plus rapide
meilleur_temps_tour_par_course =\
    fusion.groupby(['raceId', 'year'])['milliseconds'].min().reset_index()
# Fusionner à nouveau pour inclure la colonne 'name' (nom de la course)
meilleur_temps_tour_par_course =\
    pd.merge(meilleur_temps_tour_par_course, fusion[['raceId', 'name']], on='raceId',
             how='left').drop_duplicates()
# Filtrer sur l'année 2023
meilleur_temps_tour_par_course_2023 =\
    meilleur_temps_tour_par_course[meilleur_temps_tour_par_course['year'] == 2023]
# Afficher le temps le plus rapide
print("\nMeilleur temps d'un tour par course en 2023 :")
print(f"{meilleur_temps_tour_par_course_2023[['name', 'milliseconds']]}")



Meilleur temps d'un tour par course en 2023 :
                                    name  milliseconds
538121   "Bahrain Grand Prix"                    93996
539176   "Saudi Arabian Grand Prix"              91906
540118   "Australian Grand Prix"                 80235
541113   "Azerbaijan Grand Prix"                103370
542074   "Miami Grand Prix"                      89708
543212   "Monaco Grand Prix"                     75650
544725   "Spanish Grand Prix"                    76330
546037   "Canadian Grand Prix"                   74481
547352   "Austrian Grand Prix"                   67012
548705   "British Grand Prix"                    90275
549675   "Hungarian Grand Prix"                  80504
550927   "Belgian Grand Prix"                   107305
551742   "Dutch Grand Prix"                      73837
553083   "Italian Grand Prix"                    85072
554038   "Singapore Grand Prix"                  95867
555122   "Japanese Grand Prix"                   94183
556002   "Qatar Gr

Question 4 en python pur : 

In [ ]:
# Lecture et stockage des données
races_dict = {}
race_ids_2023 = set()

with open(os.path.join("donnees_formule_un", "races.csv"), 'r', encoding='utf-8') as races_file:
    reader = csv.reader(races_file)
    header_races = next(reader)
    for row in reader:
        try:
            raceId = row[0].strip()
            year = int(row[1].strip())
            if year == 2023:
                races_dict[raceId] = row
                race_ids_2023.add(raceId)
        except (IndexError, ValueError):
            continue

# Fusionner les temps et les courses pour l'année 2023
with open(os.path.join("donnees_formule_un", "lap_times.csv"), 'r', encoding='utf-8') as laps_file:
    reader = csv.reader(laps_file)
    header_laps = next(reader)

    with open("fusion_2023.csv", 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(header_laps + header_races)
        for row in reader:
            try:
                raceId = row[0].strip()
                if raceId in race_ids_2023:
                    ligne_fusionnee = row + races_dict[raceId]
                    writer.writerow(ligne_fusionnee)
            except IndexError:
                continue

# Lire et afficher les meilleurs temps par courses
meilleurs_temps = {}
with open("fusion_2023.csv", 'r', encoding='utf-8') as fusion_file:
    reader = csv.reader(fusion_file)
    header = next(reader)
    for i, row in enumerate(reader):
        try:
            nom_course = row[10].strip()
            temps_ms = int(row[5].strip())
            if nom_course not in meilleurs_temps or temps_ms < meilleurs_temps[nom_course]:
                meilleurs_temps[nom_course] = temps_ms
        except (IndexError, ValueError):
            continue


print("Meilleurs temps par course en 2023 :\n")
for nom, temps in meilleurs_temps.items():
    print(f"{nom} : {temps} ms")

Question 5 : Quel est le nombre de meilleur temps (best lap time) par pilote, en 2023 ? 

In [ ]:
# Fusion des tables
fusion= pd.merge(laps, races, on='raceId')
fusion.columns = fusion.columns.str.strip()
fusionnow = pd.merge(fusion, drivers[['driverId', 'nom_complet']], on='driverId')

# Extraire les meilleurs temps par course
meilleur_temps_tour_par_course = (
    fusionnow.loc[fusionnow.groupby(['raceId', 'year'])['milliseconds'].idxmin()]
)

# Filtrer sur l'année 2023
meilleur_temps_tour_par_course_2023 = meilleur_temps_tour_par_course[
    meilleur_temps_tour_par_course['year'] == 2023
]
# regroupe par pilote
meilleur_temps_par_pilote = (
    meilleur_temps_tour_par_course_2023.groupby('nom_complet')
    .size()
    .reset_index(name='Nombre de meilleurs temps')
    .sort_values(by='Nombre de meilleurs temps', ascending=False)
)

print("\nNombre de meilleurs temps par pilote en 2023 :")
for index, row in meilleur_temps_par_pilote.iterrows():
    print(f"{row['nom_complet']} réalise {row['Nombre de meilleurs temps']} meilleurs temps en 2023")



Nombre de meilleurs temps par pilote en 2023 :
Max Verstappen réalise 9 meilleurs temps en 2023
Lewis Hamilton réalise 4 meilleurs temps en 2023
Oscar Piastri réalise 2 meilleurs temps en 2023
Sergio Pérez réalise 2 meilleurs temps en 2023
Fernando Alonso réalise 1 meilleurs temps en 2023
George Russell réalise 1 meilleurs temps en 2023
Guanyu Zhou réalise 1 meilleurs temps en 2023
Lando Norris réalise 1 meilleurs temps en 2023
Yuki Tsunoda réalise 1 meilleurs temps en 2023


Question 6 : Quel pilote a remporté le plus de fois le circuit de Spa-Francorchamps depuis 1950 ? 

In [46]:
# Fusion des tables
fusion = pd.merge(results, races, on='raceId')
spa_gagnant = pd.merge(drivers, fusion, on='driverId')

# Filtrer la table
spa_gagnant = spa_gagnant[spa_gagnant['circuitId'] == 13] # Garder le circuit Spa-Francorchamps
spa_gagnant = spa_gagnant[spa_gagnant['positionOrder'] == 1] # Garder les gagnants

spa_gagnant = (
    spa_gagnant.groupby('nom_complet') # Regrouper par pilote
    .size()
    .reset_index(name="Nombre de victoires")  # Renommer la colonne de comptage
    .sort_values(by="Nombre de victoires", ascending=False)  # Trier par nombre de victoires décroissant
)

top2 = spa_gagnant.head(2)
num_1 = top2.iloc[0]
num_2 = top2.iloc[1]

print(f"Le pilote qui a le plus grand nombre de victoires sur le circuit de \
Spa-francorchamps est {num_1['nom_complet']} avec {num_1['Nombre de victoires']} \
victoires, suivi par {num_2['nom_complet']} \
avec {num_2['Nombre de victoires']} victoires.")

Le pilote qui a le plus grand nombre de victoires sur le circuit de Spa-francorchamps est Michael Schumacher avec 6 victoires, suivi par Ayrton Senna avec 5 victoires.


Question 7 : 

Question 8 : Quel est le temps moyen des pit stop en fonction des années ? 
(Sacahant qu'on ne dispose pas des données avant 2011)

In [ ]:

# Fusion des tables
fusion = pd.merge(races, pit, on='raceId')
fusion.columns = fusion.columns.str.strip()

pit_2023 = fusion[fusion['year'] == 2023]
pit_2020 = fusion[fusion['year'] == 2020]
pit_2016 = fusion[fusion['year'] == 2016]
pit_2013 = fusion[fusion['year'] == 2013]


# Filtre pour retirer les temps abérrants, valeurs extrêmes
def filtrer_pit_stops(df):
    q1 = df['milliseconds'].quantile(0.01)
    q99 = df['milliseconds'].quantile(0.99)
    limite_superieure = min(q99, 30000)  # max 30 sec
    return df[(df['milliseconds'] >= q1) & (df['milliseconds'] <= limite_superieure)]


pit_2023_filtre = filtrer_pit_stops(pit_2023)
pit_2020_filtre = filtrer_pit_stops(pit_2020)
pit_2016_filtre = filtrer_pit_stops(pit_2016)
pit_2013_filtre = filtrer_pit_stops(pit_2013)

# Calcul des moyennes filtrées
moy_2023_filtre = pit_2023_filtre['milliseconds'].mean()
moy_2020_filtre = pit_2020_filtre['milliseconds'].mean()
moy_2016_filtre = pit_2016_filtre['milliseconds'].mean()
moy_2013_filtre = pit_2013_filtre['milliseconds'].mean()

print(f"Le temps moyen (filtré) des pit stops en 2023 est de {moy_2023_filtre:.0f} ms.")
print(f"Il était de {moy_2020_filtre:.0f} ms en 2020, \
de {moy_2016_filtre:.0f} ms en 2016 et de {moy_2013_filtre:.0f} ms en 2013.")


Question 9 : Quel circuit a été le plus de fois concouru, depuis 1950 ?

In [75]:
# Fusion des tables
fusion = pd.merge(races, circuits, on='raceId')
fusion["name_x"] = fusion["name_x"].str.replace('"', '').str.strip()

# On regroupe par circuit et on compte le nombre de fois où il a était concurru
nb_fois_circuit = (
    fusion.groupby('name_x')
    .size()
    .reset_index(name="Nombre de fois où le circuit a été concurru")
    .sort_values(by="Nombre de fois où le circuit a été concurru", ascending=False)
)

top3 = nb_fois_circuit.head(3)
num_1 = top3.iloc[0]
num_2 = top3.iloc[1]
num_3 = top3.iloc[2]

print(f"Les circuits ayant été le plus de fois courus depuis 1950 sont le \
{num_1['name_x']} et l'{num_2['name_x']} courus \
{num_1['Nombre de fois où le circuit a été concurru']} fois, \
suivi par le {num_3['name_x']} \
couru {num_3['Nombre de fois où le circuit a été concurru']} fois.")

Les circuits ayant été le plus de fois courus depuis 1950 sont le British Grand Prix et l'Italian Grand Prix courus 75 fois, suivi par le Monaco Grand Prix couru 70 fois.


Question 9 en python pur 

Question 10 : 

Question 11 : Quel est le nombre de victoires par nation, entre 1950 et 2024 ?

In [48]:
# Fusion des tables
fusion = pd.merge(drivers, results, on="driverId")

victoire = fusion[fusion['positionOrder'] == 1]     # Garder les victoires

victoire_nation = (
    victoire.groupby('nationality')   # Regrouper par nation
    .size()
    .reset_index(name="Nombre de victoires par nation")
    .sort_values(by="Nombre de victoires par nation", ascending=False)
)

print(victoire_nation)


             nationality  Nombre de victoires par nation
6    "British"                                       312
12   "German"                                        179
5    "Brazilian"                                     101
11   "French"                                         81
9    "Dutch"                                          61
10   "Finnish"                                        57
2    "Australian"                                     43
13   "Italian"                                        43
3    "Austrian"                                       41
1    "Argentine"                                      38
19   "Spanish"                                        35
0    "American"                                       33
7    "Canadian"                                       17
16   "New Zealander"                                  12
20   "Swedish"                                        12
4    "Belgian"                                        11
18   "South African"           

Question 12 : Quel est l'âge moyen des pilotes en fonction des années ? 

In [ ]:
# Fusion des tables
fusion_1 = pd.merge(drivers, results, on='driverId')
fusion_2 = pd.merge(fusion_1, races, on='raceId')

drivers_unique = fusion_2.drop_duplicates(subset=["driverId"]) # Vérifier les doublons

# Filtrer sur l'année 2023
driver_unique_2023 = drivers_unique[drivers_unique['year'] == 2023].copy()
driver_unique_2023.columns = driver_unique_2023.columns.str.strip()

# Convertir la date de naissance en format datetime pour calculer l'âge
driver_unique_2023['dob'] = driver_unique_2023['dob'].str.strip().str.replace('"', '')
driver_unique_2023['dob'] = pd.to_datetime(driver_unique_2023['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_2023.loc[:, 'dob'] = pd.to_datetime(driver_unique_2023['dob'])
driver_unique_2023[driver_unique_2023['dob'].isna()]

aujourdhui = datetime.now()
driver_unique_2023.loc[:, 'age'] = driver_unique_2023['dob'].apply(
    lambda x: aujourdhui.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)

age_moyen = driver_unique_2023['age'].mean()   # Calcul de l'âge moyen
print(f"L'âge moyen des pilotes en 2023 est de {age_moyen:.2f} ans")


# Idem pour l'année 2000 :
driver_unique_2000 = drivers_unique[drivers_unique['year'] == 2000].copy()
driver_unique_2000.columns = driver_unique_2000.columns.str.strip()
driver_unique_2000['dob'] = driver_unique_2000['dob'].str.strip().str.replace('"', '')
driver_unique_2000['dob'] = pd.to_datetime(driver_unique_2000['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_2000.loc[:, 'dob'] = pd.to_datetime(driver_unique_2000['dob'])
driver_unique_2000[driver_unique_2000['dob'].isna()]
aujourdhui_2000 = datetime(2000, 1, 1)  # Date du 1er janvier 2000
driver_unique_2000.loc[:, 'age'] = driver_unique_2000['dob'].apply(
    lambda x: aujourdhui_2000.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)
age_moyen = driver_unique_2000['age'].mean()
print(f"L'âge moyen des pilotes en 2000 est de {age_moyen:.2f} ans")


# Idem pour l'année 1980 :
driver_unique_1980 = drivers_unique[drivers_unique['year'] == 1980].copy()
driver_unique_1980.columns = driver_unique_1980.columns.str.strip()
driver_unique_1980['dob'] = driver_unique_1980['dob'].str.strip().str.replace('"', '')
driver_unique_1980['dob'] = pd.to_datetime(driver_unique_1980['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_1980.loc[:, 'dob'] = pd.to_datetime(driver_unique_1980['dob'])
driver_unique_1980[driver_unique_1980['dob'].isna()]
aujourdhui_1980 = datetime(1980, 1, 1)  # Date du 1er janvier 1980
driver_unique_1980.loc[:, 'age'] = driver_unique_1980['dob'].apply(
    lambda x: aujourdhui_1980.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)
age_moyen = driver_unique_1980['age'].mean()
print(f"L'âge moyen des pilotes en 1980 est de {age_moyen:.2f} ans")

# Idem pour l'année 1950 :
driver_unique_1950 = drivers_unique[drivers_unique['year'] == 1950].copy()
driver_unique_1950.columns = driver_unique_1950.columns.str.strip()
driver_unique_1950['dob'] = driver_unique_1950['dob'].str.strip().str.replace('"', '')
driver_unique_1950['dob'] = pd.to_datetime(driver_unique_1950['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_1950.loc[:, 'dob'] = pd.to_datetime(driver_unique_1950['dob'])
driver_unique_1950[driver_unique_1950['dob'].isna()]
aujourdhui_1950 = datetime(1950, 1, 1)  # Date du 1er janvier 1950
driver_unique_1950.loc[:, 'age'] = driver_unique_1950['dob'].apply(
    lambda x: aujourdhui_1950.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)
age_moyen = driver_unique_1950['age'].mean()
print(f"L'âge moyen des pilotes en 1950 est de {age_moyen:.2f} ans")